In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="3"

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import glob
from torch.utils.data import Dataset
import torchvision.datasets as datasets


import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [3]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices
            
        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        dataset_type = type(dataset)
        if dataset_type is torchvision.datasets.MNIST:
            return dataset.train_labels[idx].item()
        elif dataset_type is torchvision.datasets.ImageFolder:
            return dataset.imgs[idx][1]
        else:
            raise NotImplementedError
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [4]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
    
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    
def validate(val_loader, model, criterion):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for i, (images, target, _) in enumerate(val_loader):
            images = images.cuda()
            target = target.cuda()

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            if i % 10 == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


In [5]:
best_acc1 = 0

exp = 'exp1'

if exp == 'exp1':
    num_classes = 14
else:
    num_classes = 13
    
num_epochs = 20
classifier = models.resnet50(pretrained=True)

for param in classifier.parameters():
    param.requires_grad = False
classifier.fc = nn.Linear(2048, num_classes)

classifier.cuda()

optimizer = torch.optim.Adam(classifier.parameters(), 1e-3,
#                             momentum=0.9,
                            weight_decay=1e-5)


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    './data/%s'%exp,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=False, pin_memory=True, 
    sampler=ImbalancedDatasetSampler(train_dataset))

val_dataset = ImageFolderWithPaths('./test_data/%s'%exp, transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        normalize,
    ]))
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=128, shuffle=False,
    pin_memory=True, drop_last=False)

criterion = nn.CrossEntropyLoss()

In [5]:
# import glob
# from PIL import Image
# paths = glob.glob('./data/*/*/*', recursive=True)
# # paths = glob.glob('./Google-Image-Downloader/fork/*', recursive=True)
# failed = []
# for path in paths:
#     try:
#         z = Image.open(path)
#     except:
#         failed.append(path)
# print(failed)
# for i in failed:
#     try:
#         os.remove(i)
#     except:
#         continue

In [6]:
for epoch in range(num_epochs):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))
    
    if epoch == num_epochs // 2:
        for param_group in optimizer.param_groups:
            param_group["lr"] /= 10


    for i, (im, target) in enumerate(train_loader):
        classifier.train()
        im = im.cuda()
        target = target.cuda()
        
        output = classifier(im)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), im.size(0))
        top1.update(acc1[0], im.size(0))
        top5.update(acc5[0], im.size(0))

        if i % 10 == 0:
            progress.display(i)
        
    acc1 = validate(val_loader, classifier, criterion)
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)
    if is_best:
        torch.save(classifier.state_dict(), 'model_%s.pth'%exp)

Epoch: [0][ 0/98]	Loss 2.7016e+00 (2.7016e+00)	Acc@1   7.81 (  7.81)	Acc@5  42.19 ( 42.19)
Epoch: [0][10/98]	Loss 1.3840e+00 (1.9085e+00)	Acc@1  75.78 ( 54.19)	Acc@5  97.66 ( 82.88)
Epoch: [0][20/98]	Loss 7.6161e-01 (1.4331e+00)	Acc@1  82.81 ( 68.08)	Acc@5  98.44 ( 90.10)
Epoch: [0][30/98]	Loss 5.2857e-01 (1.1606e+00)	Acc@1  87.50 ( 74.45)	Acc@5  98.44 ( 92.94)
Epoch: [0][40/98]	Loss 5.0018e-01 (1.0020e+00)	Acc@1  88.28 ( 77.44)	Acc@5  99.22 ( 94.42)
Epoch: [0][50/98]	Loss 4.5108e-01 (8.9473e-01)	Acc@1  90.62 ( 79.47)	Acc@5  98.44 ( 95.25)
Epoch: [0][60/98]	Loss 3.9874e-01 (8.1399e-01)	Acc@1  85.16 ( 80.98)	Acc@5  99.22 ( 95.84)
Epoch: [0][70/98]	Loss 3.6757e-01 (7.5274e-01)	Acc@1  89.06 ( 82.13)	Acc@5 100.00 ( 96.29)
Epoch: [0][80/98]	Loss 3.7467e-01 (7.0463e-01)	Acc@1  88.28 ( 83.13)	Acc@5  98.44 ( 96.60)
Epoch: [0][90/98]	Loss 4.4711e-01 (6.6793e-01)	Acc@1  87.50 ( 83.83)	Acc@5  96.09 ( 96.82)
Test: [ 0/10]	Loss 8.6590e-01 (8.6590e-01)	Acc@1  74.22 ( 74.22)	Acc@5  91.41 ( 91.41)
 * 

Epoch: [8][ 0/98]	Loss 2.1208e-01 (2.1208e-01)	Acc@1  90.62 ( 90.62)	Acc@5 100.00 (100.00)
Epoch: [8][10/98]	Loss 4.1439e-01 (2.7266e-01)	Acc@1  87.50 ( 91.34)	Acc@5  98.44 ( 99.50)
Epoch: [8][20/98]	Loss 2.1323e-01 (2.4939e-01)	Acc@1  91.41 ( 92.22)	Acc@5 100.00 ( 99.63)
Epoch: [8][30/98]	Loss 2.9323e-01 (2.5227e-01)	Acc@1  90.62 ( 92.16)	Acc@5  99.22 ( 99.42)
Epoch: [8][40/98]	Loss 2.0982e-01 (2.5922e-01)	Acc@1  91.41 ( 91.62)	Acc@5 100.00 ( 99.39)
Epoch: [8][50/98]	Loss 2.9723e-01 (2.5616e-01)	Acc@1  89.06 ( 91.74)	Acc@5  98.44 ( 99.45)
Epoch: [8][60/98]	Loss 2.1883e-01 (2.5796e-01)	Acc@1  92.19 ( 91.66)	Acc@5 100.00 ( 99.39)
Epoch: [8][70/98]	Loss 1.9968e-01 (2.5766e-01)	Acc@1  91.41 ( 91.74)	Acc@5  99.22 ( 99.37)
Epoch: [8][80/98]	Loss 1.8665e-01 (2.5168e-01)	Acc@1  94.53 ( 91.88)	Acc@5  99.22 ( 99.43)
Epoch: [8][90/98]	Loss 2.3687e-01 (2.5266e-01)	Acc@1  92.97 ( 91.85)	Acc@5  98.44 ( 99.40)
Test: [ 0/10]	Loss 8.5644e-01 (8.5644e-01)	Acc@1  79.69 ( 79.69)	Acc@5  92.19 ( 92.19)
 * 

 * Acc@1 61.695 Acc@5 89.717
Epoch: [16][ 0/98]	Loss 1.7378e-01 (1.7378e-01)	Acc@1  94.53 ( 94.53)	Acc@5 100.00 (100.00)
Epoch: [16][10/98]	Loss 1.9785e-01 (2.4838e-01)	Acc@1  92.19 ( 92.54)	Acc@5 100.00 ( 99.72)
Epoch: [16][20/98]	Loss 1.7631e-01 (2.3495e-01)	Acc@1  95.31 ( 92.75)	Acc@5  98.44 ( 99.70)
Epoch: [16][30/98]	Loss 2.5805e-01 (2.2893e-01)	Acc@1  90.62 ( 92.79)	Acc@5  98.44 ( 99.60)
Epoch: [16][40/98]	Loss 2.7641e-01 (2.3074e-01)	Acc@1  88.28 ( 92.61)	Acc@5 100.00 ( 99.54)
Epoch: [16][50/98]	Loss 2.4916e-01 (2.3119e-01)	Acc@1  90.62 ( 92.68)	Acc@5  99.22 ( 99.49)
Epoch: [16][60/98]	Loss 2.6526e-01 (2.3350e-01)	Acc@1  90.62 ( 92.43)	Acc@5  98.44 ( 99.51)
Epoch: [16][70/98]	Loss 2.9491e-01 (2.3841e-01)	Acc@1  90.62 ( 92.39)	Acc@5 100.00 ( 99.54)
Epoch: [16][80/98]	Loss 2.2764e-01 (2.3815e-01)	Acc@1  92.97 ( 92.40)	Acc@5 100.00 ( 99.55)
Epoch: [16][90/98]	Loss 3.1011e-01 (2.4026e-01)	Acc@1  89.84 ( 92.38)	Acc@5  99.22 ( 99.51)
Test: [ 0/10]	Loss 9.0149e-01 (9.0149e-01)	Acc@1  7

In [11]:
out_dict = {}
idx2class = train_dataset.classes
classifier.load_state_dict(torch.load('model_%s.pth'%exp))
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f')

classifier.eval()
with torch.no_grad():
    for i, (images, target, path) in enumerate(val_loader):
        images = images.cuda()
        target = target.cuda()

        # compute output
        output = classifier(images)
        val, idx = F.softmax(output, 1).sort(dim=1, descending=True)  # [B, num_classes]
        for i in range(idx.size(0)):
            labels = ["%s: %.3f" % (idx2class[idx[i][j]], val[i][j].item()) for j in range(num_classes)]
            labels = "Actual: %s, "%idx2class[target[i]] + ", ".join(labels)
            out_dict[path[i]] = labels


        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

    # TODO: this should also be done with the ProgressMeter
    print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

 * Acc@1 64.286 Acc@5 88.069


In [12]:
import csv
print(len(out_dict))

w = csv.writer(open("nocrop_output_%s.csv"%exp, "w"))
for key, val in out_dict.items():
    w.writerow([key, val])

1274


In [9]:
# import os
# import shutil

# in_dir = './Images/coco_images'
# out_dir = './exp2/test_data'

# idx2class  = {1:'person',3:'car',5:'airplane',16:'bird',18:'dog',22:'elephant',23:'bear',24:'zebra',48:'fork',
#               49:'knife',51:'bowl',53:'apple',62:'chair',67:'table'}
 
# os.makedirs(out_dir, exist_ok=True)
# for k,v in idx2class.items():
#     out = os.path.join(out_dir, v)
#     os.makedirs(out, exist_ok=True)


# csv_dict = {}
# f =  open("Book1.csv")
# lis = [line.strip().split(',') for line in f]        # create a list of lists
# headers = lis.pop(0)   
# for i, header in enumerate(headers):
#     csv_dict[header] = i

# for item in lis:
#     name = item[csv_dict['filename']][1:-1]
#     try:
#         category = int(item[csv_dict['category_id']])
#     except:
#         category = item[csv_dict['category_id']]
#         break
#     input_name = os.path.join(in_dir, name)
#     output_name = os.path.join(out_dir, idx2class[category], name)
#     try:
#         shutil.move(input_name, output_name)
#     except:
#         continue